In [5]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [9]:
# set up dataframes
providers = pd.read_csv('')



'''

providers = pd.read_csv('Medicare_Provider_Util_Payment_PUF_CY2017.csv') 
outpatient = pd.read_csv('../data/MUP_OHP_R19_P04_V10_D17_APC_Provider.csv') 
classification = pd.read_csv('../data/508-Compliant-Version-of-2020_january_web_addendum_b.12312019.csv') 
classification2 = pd.read_csv('../data/2020_january_web_addendum_b.12312019.csv') 
cbsa = pd.read_csv('../data/ZIP_CBSA_032020.csv') 

'''

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-f064332495ab>", line 2, in <module>
    providers = pd.read_csv('Medicare_Provider_Util_Payment_PUF_CY2017.csv')
  File "/opt/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 685, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "/opt/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 457, in _read
    parser = TextFileReader(fp_or_buf, **kwds)
  File "/opt/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 895, in __init__
    self._make_engine(self.engine)
  File "/opt/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 1135, in _make_engine
    self._engine = CParserWrapper(self.f, **self.options)
  File "/opt/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 1917, in __init_

FileNotFoundError: [Errno 2] File b'Medicare_Provider_Util_Payment_PUF_CY2017.csv' does not exist: b'Medicare_Provider_Util_Payment_PUF_CY2017.csv'